# Segmenting and Clustering Neighborhoods in Toronto
In the following program there is a geographic analysis of Toronto city. It is divided in three parts:
1. Web scraping: obtain the database of the neighborhoods needed for the analysis
2. Coordinates: obtain the coordinates of each neighborhood and venue. Create a dataframe with the characteristics of each neighborhood according to the frequency of the different categories of venues. 
3. K-means to cluster neighborhoods: use the k-means algorithm to cluster the neighborhoods into 5 groups as of their characterization in the previous step. Display the results.

## 1. Web Scraping

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

From the wikipedia page extract all the dataframes:

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df = pd.read_html(url, header=0)
len(df)

3

Select the dataframe of interest and display the first 5 values:

In [3]:
print(len(df[0]["Neighborhood"].unique()))
df_t1 = df[0]
#type(df_t1)
print(df_t1.shape)
df_t1.head()

100
(180, 3)


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Delete the rows with "Not assigned" for the boroughs. Combine the neighborhood names according to its postal code. One thing to highlight is that there are less unique neighborhood values (i.e. less neighborhood names) than unique postal code values. This means that: i) one neighborhood has two postal codes; or ii) there are different neighborhoods with the same name. _**For this reason, great part of the data analysis is done with the postal code identifier instead of the neighborhood name**_:

In [48]:
df_t2 = df_t1[df_t1["Borough"] != "Not assigned"]
print("The number of unique postal codes:",len(df_t2["Postal Code"].unique())) 
print("The number of unique neighborhoods:",len(df_t2["Neighborhood"].unique()))
df_t2.sort_values("Postal Code", ascending=True)
df_t2.reset_index(inplace=True, drop=True)
df_t2.head()

The number of unique postal codes: 103
The number of unique neighborhoods: 99


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Here I verify if there is any "Not assigned" value left in Borough or Neighborhood:

In [49]:
if "Not assigned" in df_t2["Borough"].unique():
    print("clean")
else: print("ok")

if "Not assigned" in df_t2["Neighborhood"].unique():
    print("clean")
else: print("ok")

ok
ok


In [50]:
print("The number of rows of the dataset is:",df_t2.shape[0],"and of columns:",df_t2.shape[1])

The number of rows of the dataset is: 103 and of columns: 3


## 2. Coordinates
In this section I add the coordinates information to each neighborhood, according to its postal code.

In this first try is the code necessary to obtain the latitude and longitude from the `geocoder` library.

In [7]:
#! pip install geocoder
#import geocoder
#column_names = ["Postal Code","Borough","Neighborhood","Latitude","Longitude"]
#nh = pd.DataFrame(columns=column_names)
#lat_lng_coords = None
#i = 0
#h= 0
#for post, bor, nei in zip(df_t2["Postal Code"],df_t2["Borough"],df_t2["Neighborhood"]):
#    lat_lng_coords = None
#    h=0
#    while(lat_lng_coords is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(post))
#        lat_lng_coords = g.latlng
#        h+=1
#        print(h)
#        if h==50: 
#            lat_lng_coords = ["None","None"]
            
#    latitude = lat_lng_coords[0]
#     longitude = lat_lng_coords[1]
#    nh = nh.append({'Postal Code': post,
#                   'Borough': bor,
#                   'Neighborhood': nei,
#                   'Latitude':latitude,
#                   'Longitude':longitude}, ignore_index = True)
#    print(i)
#    i+=1

After many iterations, this api wasn't able to produce the information needed, so I used an external database:

In [8]:
df_lat_lng = pd.read_csv("Geospatial_Coordinates.csv")
print(df_lat_lng.shape)
df_lat_lng.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Here I merge dataframes by "Postal Code":

In [9]:
df_t3 = pd.merge(left=df_t2, right=df_lat_lng, how="left", left_on="Postal Code", right_on = "Postal Code")
print(df_t3.shape)
#print(df_t3["Neighborhood"].unique())
df_t3.head()

(103, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Here is use the `geocode` library to get the coordinates of Toronto City:

In [10]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print("The geographical coordinate of Toronto asre {}, {}.".format(latitude,longitude))

The geographical coordinate of Toronto asre 43.6534817, -79.3839347.


The folowing script yields the map of Toronto with the location of each neighborhood (signaled by the markers):

In [13]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df_t3['Latitude'], df_t3['Longitude'], df_t3['Borough'], df_t3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='midnightblue',
        fill=True,
        fill_color='maroon',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [14]:
CLIENT_ID = 'SECRET' # your Foursquare ID
CLIENT_SECRET = 'SECRET' # your Foursquare Secret
VERSION = '20200606' # Foursquare API version

Generate the function that makes a list with the name and coordinates of the nearby locations of each postal code (with limit of 200 locations for each neighborhood and a radius of 1km):

In [33]:
LIMIT = 200
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except: continue
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
toronto_venues = getNearbyVenues(names=df_t3["Postal Code"],
                                latitudes= df_t3["Latitude"],
                                longitudes=df_t3["Longitude"])
toronto_venues.shape

(4881, 7)

In the following dataframe is presented the first 5 venues found with the **foursquare API**:

In [35]:
print(toronto_venues.shape)
toronto_venues.head()


(4881, 7)


,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,M3A,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
3,M3A,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store
4,M3A,43.753259,-79.329656,High Street Fish & Chips,43.745260,-79.324949,Fish & Chips Shop


In [36]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 331 uniques categories.


The following script generates a table that identifies each venue according to its category (there are 331 different categories):

In [37]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
cols = list(toronto_onehot.columns)

# add neighborhood column back to dataframe
toronto_onehot['Postal Code'] = toronto_venues['Postal Code'] 

fixed_columns = [toronto_onehot.columns[-1]]+ list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [39]:
print(toronto_onehot.shape)
print(len(toronto_onehot["Postal Code"].unique()))
toronto_onehot.head()

(4881, 332)
101


,Postal Code,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,

This dataframe contains the mean of the frequency of occurrence of each category:

In [40]:
toronto_g = toronto_onehot.groupby("Postal Code").mean().reset_index()
print(toronto_g.shape)
toronto_g.head()

(101, 332)


,Postal Code,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,

Here are the top 5 most frequent venue categories for each postal code location:

In [41]:
num_top_venues = 5

for hood in toronto_g['Postal Code']:
    print("----"+hood+"----")
    temp = toronto_g[toronto_g['Postal Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1C----
                venue  freq
0                Park   0.2
1      Breakfast Spot   0.2
2        Burger Joint   0.2
3  Italian Restaurant   0.2
4          Playground   0.2


----M1E----
                  venue  freq
0           Pizza Place  0.12
1  Fast Food Restaurant  0.08
2                  Bank  0.08
3            Restaurant  0.08
4           Coffee Shop  0.08


----M1G----
                  venue  freq
0                  Park  0.22
1           Coffee Shop  0.22
2  Fast Food Restaurant  0.11
3              Pharmacy  0.11
4    Chinese Restaurant  0.11


----M1H----
               venue  freq
0             Bakery  0.10
1        Coffee Shop  0.10
2               Bank  0.06
3  Indian Restaurant  0.06
4        Gas Station  0.06


----M1J----
                  venue  freq
0        Ice Cream Shop  0.15
1         Bowling Alley  0.08
2        Sandwich Place  0.08
3  Fast Food Restaurant  0.08
4           Coffee Shop  0.08


----M1K----
                venue  freq
0  Chinese Restauran

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

This dataframe shows the top 20 mosto common venue category for each neighborhood:

In [43]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postal Code'] = toronto_g['Postal Code']

for ind in np.arange(toronto_g.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_g.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,M1C,Italian Restaurant,Breakfast Spot,Playground,Burger Joint,Park,Zoo,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Field,Fast Food Restaurant,Drugstore,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market
1,M1E,Pizza Place,Coffee Shop,Fast Food Restaurant,Restaurant,Bank,Filipino Restaurant,Pharmacy,Smoothie Shop,Food & Drink Shop,Sports Bar,Beer Store,Liquor Store,Sandwich Place,Greek Restaurant,Moving Target,Chinese Restaurant,Fried Chicken Joint,Discount Store,Supermarket,Ethiopian Restaurant
2,M1G,Park,Coffee Shop,Chinese Restaurant,Indian Restaurant,Fast Food Restaurant,Pharmacy,Mobile Phone Shop,Dance Studio,Farmers Market,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Creperie,Drugstore,Field,Filipino Restaurant,Fireworks Store
3,M1H,Bakery,Coffee Shop,Bank,Indian Restaurant,Gas Station,Grocery Store,Gym / Fitness Center,Burger Joint,Fast Food Restaurant,Pizza Place,Sporting Goods Shop,Fried Chicken Joint,Caribbean Restaurant,Music Store,Chinese Restaurant,Thai Restaurant,Bus Line,Wings Joint,Athletics & Sports,Pharmacy
4,M1J,Ice Cream Shop,Coffee Shop,Train Station,Restaurant,Fast Food Restaurant,Japanese Restaurant,Bowling Alley,Convenience Store,Sandwich Place,Pizza Place,Women's Store,Grocery Store,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Drugstore,Ethiopian Restaurant,Event Space,Zoo,Falafel Restaurant


## 3. K-means to cluster Neighborhoods
The following scripts runs a *k-means* algorithm to cluster in five groups all the neighborhoods according to the frequency of the different categories of venues in each neighborhood:

In [44]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_g.drop('Postal Code', axis = 1)
toronto_grouped_clustering.head()
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
check1 = list(kmeans.labels_)
len(check1)

101

In [45]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_t3

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Postal Code'), on='Postal Code')
print(toronto_merged["Cluster Labels"].unique())
toronto_merged.head() # check the last columns!
toronto_merged.replace("nan", np.nan, inplace = True)
toronto_merged.dropna(inplace=True)
print(toronto_merged["Cluster Labels"].unique())

[ 2.  3.  0. nan  1.  4.]
[2. 3. 0. 1. 4.]


### Clustered Neighborhoods Map
This map shows the 5 different clusters of neighborhoods, indicating which neighborhoods are more similar between them:

In [47]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#print(colors_array)
rainbow = [colors.rgb2hex(i) for i in colors_array]
#print(rainbow)
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    #print(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### __*Cluster 1*__

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
5,Etobicoke,0.0,Pharmacy,Convenience Store,Bank,Bakery,Playground,Golf Course,Shopping Mall,Park,Café,Grocery Store,Electronics Store,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Event Space,Ethiopian Restaurant,Doner Restaurant,Falafel Restaurant,Farm,Farmers Market
12,Scarborough,0.0,Italian Restaurant,Breakfast Spot,Playground,Burger Joint,Park,Zoo,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Field,Fast Food Restaurant,Drugstore,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market
50,North York,0.0,Italian Restaurant,Bank,Bakery,Medical Center,Pizza Place,Shopping Mall,Electronics Store,Pharmacy,Arts & Crafts Store,Park,Zoo,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Event Space,Farm,Falafel Restaurant,Donut Shop,Farmers Market,Fast Food Restaurant
57,North York,0.0,Golf Course,Storage Facility,Gas Station,Park,Discount Store,Intersection,Bakery,Convenience Store,Filipino Restaurant,Fish Market,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Flea Market,Event Space,Fireworks Store,Falafel Restaurant,Farm,Drugstore
58,Scarborough,0.0,Park,Thai Restaurant,Farm,Diner,Skating Rink,Gym,Café,Gym Pool,Restaurant,General Entertainment,Ice Cream Shop,College Stadium,Convenience Store,Farmers Market,Fireworks Store,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
66,North York,0.0,Park,Restaurant,Coffee Shop,Gym,Grocery Store,French Restaurant,Gas Station,Intersection,Dentist's Office,Golf Course,Business Service,Tennis Court,Convenience Store,Playground,Dog Run,Dumpling Restaurant,Electronics Store,Drugstore,Eastern European Restaurant,Zoo
88,Etobicoke,0.0,Park,Restaurant,Indian Restaurant,Skating Rink,Italian Restaurant,Fried Chicken Joint,Grocery Store,Bakery,Café,Pharmacy,Fast Food Restaurant,Pub,Liquor Store,American Restaurant,Pizza Place,Mexican Restaurant,Dessert Shop,Event Space,Electronics Store,Ethiopian Restaurant
101,Etobicoke,0.0,Italian Restaurant,Park,Shopping Mall,Eastern European Restaurant,Ice Cream Shop,Bus Stop,Zoo,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Farm,Donut Shop,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop


### __*Cluster 2*__

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
45,North York,1.0,Park,Pool,Zoo,Farm,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Doner Restaurant,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market


### _**Cluster 3**_

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,North York,2.0,Park,Convenience Store,Bus Stop,Pharmacy,Shopping Mall,Food & Drink Shop,Pizza Place,Café,Road,Supermarket,Shop & Service,Fish & Chips Shop,Caribbean Restaurant,Laundry Service,Chinese Restaurant,Fast Food Restaurant,Discount Store,Tennis Court,Cosmetics Shop,Train Station
8,East York,2.0,Brewery,Pizza Place,Gym / Fitness Center,Fast Food Restaurant,Bank,Bakery,Rock Climbing Spot,Coffee Shop,Café,Intersection,Athletics & Sports,Gastropub,Home Service,Soccer Stadium,Construction & Landscaping,Pharmacy,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
10,North York,2.0,Grocery Store,Fast Food Restaurant,Pizza Place,Metro Station,Furniture / Home Store,Discount Store,Gas Station,Coffee Shop,Playground,Restaurant,Mediterranean Restaurant,Latin American Restaurant,Japanese Restaurant,Department Store,Park,Bank,Pub,Pet Store,Asian Restaurant,Gym
11,Etobicoke,2.0,Park,Pizza Place,Mexican Restaurant,Gym,Restaurant,Coffee Shop,Clothing Store,Fish & Chips Shop,Bank,Grocery Store,Convenience Store,Hotel,Hardware Store,Electronics Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
16,York,2.0,Pizza Place,Coffee Shop,Convenience Store,Grocery Store,Mexican Restaurant,Trail,Sushi Restaurant,Sandwich Place,Korean Restaurant,Gastropub,Middle Eastern Restaurant,Bank,Optical Shop,Bagel Shop,Gift Shop,Italian Restaurant,Frozen Yogurt Shop,Dance Studio,Hockey Arena,Soccer Stadium
18,Scarborough,2.0,Pizza Place,Coffee Shop,Fast Food Restaurant,Restaurant,Bank,Filipino Restaurant,Pharmacy,Smoothie Shop,Food & Drink Shop,Sports Bar,Beer Store,Liquor Store,Sandwich Place,Greek Restaurant,Moving Target,Chinese Restaurant,Fried Chicken Joint,Discount Store,Supermarket,Ethiopian Restaurant
21,York,2.0,Pharmacy,Park,Bus Stop,Fast Food Restaurant,Beer Store,Bank,Bakery,Mexican Restaurant,Falafel Restaurant,Discount Store,Grocery Store,Japanese Restaurant,Coffee Shop,Hostel,Bus Line,Pizza Place,Cosmetics Shop,Women's Store,Electronics Store,Field
22,Scarborough,2.0,Park,Coffee Shop,Chinese Restaurant,Indian Restaurant,Fast Food Restaurant,Pharmacy,Mobile Phone Shop,Dance Studio,Farmers Market,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Creperie,Drugstore,Field,Filipino Restaurant,Fireworks Store
26,Scarborough,2.0,Bakery,Coffee Shop,Bank,Indian Restaurant,Gas Station,Grocery Store,Gym / Fitness Center,Burger Joint,Fast Food Restaurant,Pizza Place,Sporting Goods Shop,Fried Chicken Joint,Caribbean Restaurant,Music Store,Chinese Restaurant,Thai Restaurant,Bus Line,Wings Joint,Athletics & Sports,Pharmacy
27,North York,2.0,Pharmacy,Coffee Shop,Park,Convenience Store,Grocery Store,Fast Food Restaurant,Shopping Mall,Chinese Restaurant,Sandwich Place,Korean Restaurant,Bank,Restaurant,Residential Building (Apartment / Condo),Bakery,Recreation Center,Pizza Place,Ice Cream Shop,Eastern European Restaurant,Doner Restaurant,Donut Shop


### _**Cluster 4**_

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
1,North York,3.0,Coffee Shop,Hockey Arena,Sporting Goods Shop,Portuguese Restaurant,Men's Store,Lounge,Park,Golf Course,Intersection,Gym / Fitness Center,Zoo,Ethiopian Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Event Space,Doner Restaurant,Farm
2,Downtown Toronto,3.0,Coffee Shop,Pub,Park,Diner,Theater,Café,Bakery,Breakfast Spot,Italian Restaurant,Restaurant,Thai Restaurant,Sushi Restaurant,Performing Arts Venue,Gastropub,Indian Restaurant,Gym / Fitness Center,Dessert Shop,Mediterranean Restaurant,French Restaurant,Karaoke Bar
3,North York,3.0,Fast Food Restaurant,Furniture / Home Store,Clothing Store,Coffee Shop,Vietnamese Restaurant,Dessert Shop,Restaurant,Fried Chicken Joint,Sushi Restaurant,Café,Men's Store,Fireworks Store,Bank,Miscellaneous Shop,Korean Restaurant,Cheese Shop,Event Space,Boutique,Bowling Alley,Seafood Restaurant
4,Downtown Toronto,3.0,Coffee Shop,Park,Café,Ramen Restaurant,Italian Restaurant,Japanese Restaurant,Sushi Restaurant,Burger Joint,Gay Bar,Thai Restaurant,Bookstore,Ice Cream Shop,Cosmetics Shop,Gastropub,Dance Studio,Yoga Studio,Bubble Tea Shop,Clothing Store,Pizza Place,Middle Eastern Restaurant
7,North York,3.0,Coffee Shop,Japanese Restaurant,Pizza Place,Burger Joint,Bank,Bar,Shop & Service,Supermarket,Liquor Store,Thai Restaurant,Caribbean Restaurant,Restaurant,Diner,Salad Place,Greek Restaurant,Asian Restaurant,Gym,Breakfast Spot,Spa,Café
9,Downtown Toronto,3.0,Coffee Shop,Japanese Restaurant,Gastropub,Italian Restaurant,Ramen Restaurant,Restaurant,Clothing Store,Theater,Hotel,Poke Place,Plaza,Pizza Place,Diner,Café,Cosmetics Shop,Burrito Place,Creperie,Gym,Middle Eastern Restaurant,Shopping Mall
13,North York,3.0,Restaurant,Coffee Shop,Gym,Japanese Restaurant,Supermarket,Beer Store,Salon / Barbershop,Asian Restaurant,Intersection,Italian Restaurant,Bank,History Museum,Clothing Store,New American Restaurant,Sporting Goods Shop,Sushi Restaurant,Bike Shop,Sandwich Place,Chinese Restaurant,Burger Joint
14,East York,3.0,Coffee Shop,Pizza Place,Park,Thai Restaurant,Athletics & Sports,Sandwich Place,Greek Restaurant,Beer Store,Liquor Store,Convenience Store,Restaurant,Farmers Market,Pharmacy,Skating Rink,Café,Bus Stop,Pastry Shop,Pub,Ice Cream Shop,Video Store
15,Downtown Toronto,3.0,Coffee Shop,Café,Restaurant,Japanese Restaurant,Italian Restaurant,Gastropub,Plaza,Hotel,Bookstore,Seafood Restaurant,Furniture / Home Store,Bakery,Tea Room,Clothing Store,Theater,Creperie,Gym,Cosmetics Shop,Poke Place,Middle Eastern Restaurant
17,Etobicoke,3.0,Coffee Shop,Fish & Chips Shop,Pharmacy,Liquor Store,Beer Store,Shopping Mall,Grocery Store,Café,College Rec Center,Pet Store,Intersection,Farmers Market,IT Services,Pizza Place,Gas Station,Farm,Drugstore,Fast Food Restaurant,Falafel Restaurant,Field


### _**Cluster 5**__

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
53,North York,4.0,Vietnamese Restaurant,Food Truck,Baseball Field,Farmers Market,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Donut Shop,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
